# 1. 데이터 준비하기

In [4]:
import zipfile
import os

def unzip_file(zip_file_path, extract_to_dir):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_dir)
        print(f"Extracted all files to {extract_to_dir}")

# Example usage
zip_file_path = './casting_defect.zip'
extract_to_dir = './'

# Ensure the extraction directory exists
os.makedirs(extract_to_dir, exist_ok=True)

unzip_file(zip_file_path, extract_to_dir)

Extracted all files to ./


In [6]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

import tf_keras as keras

data_dir = "./casting_defect"

def build_dataset(subset):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            validation_split=0.20
    )

    # Generate batches of image data and labels
    return datagen.flow_from_directory(
        directory=data_dir,
        target_size=(224,224),
        batch_size=32,
        class_mode='binary',
        subset=subset,  # Specify 'training' or 'validation' subset
        seed=123  # Seed for reproducibility
    )

# Build training and validation datasets
train_ds = build_dataset("training")
val_ds = build_dataset("validation")

# Capture class names from the directory structure
class_names = train_ds.class_indices

# Retrieve dataset sizes
train_size = train_ds.samples
valid_size = val_ds.samples

Found 1041 images belonging to 2 classes.
Found 259 images belonging to 2 classes.


# 2. 모델 정의하기

In [7]:
model_handle = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
print("Building model with", model_handle)
model = keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    keras.layers.InputLayer(input_shape= (224,224,3)),
    hub.KerasLayer(model_handle, trainable=False),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation='sigmoid')
])
model.build((None,224,224,3))
model.summary()

Building model with https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 module_wrapper (ModuleWrap  (None, 1280)              2257984   
 per)                                                            
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2259265 (8.62 MB)
Trainable params: 1281 (5.00 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


# 3. 학습 & 검증 하기

In [8]:
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'])

In [12]:
steps_per_epoch = train_size // 32
validation_steps = valid_size // 32
hist = model.fit(
    train_ds,
    epochs=10, steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps).history

Epoch 1/10
32/32 [==============================] - 3s 103ms/step - loss: 0.1413 - accuracy: 0.9554 - val_loss: 0.1379 - val_accuracy: 0.9648
Epoch 2/10
32/32 [==============================] - 4s 115ms/step - loss: 0.1349 - accuracy: 0.9633 - val_loss: 0.1565 - val_accuracy: 0.9383
Epoch 3/10
32/32 [==============================] - 3s 110ms/step - loss: 0.1312 - accuracy: 0.9633 - val_loss: 0.1450 - val_accuracy: 0.9648
Epoch 4/10
32/32 [==============================] - 4s 119ms/step - loss: 0.1337 - accuracy: 0.9584 - val_loss: 0.1464 - val_accuracy: 0.9427
Epoch 5/10
32/32 [==============================] - 3s 83ms/step - loss: 0.1373 - accuracy: 0.9514 - val_loss: 0.1435 - val_accuracy: 0.9559
Epoch 6/10
32/32 [==============================] - 3s 98ms/step - loss: 0.1229 - accuracy: 0.9643 - val_loss: 0.1328 - val_accuracy: 0.9471
Epoch 7/10
32/32 [==============================] - 3s 81ms/step - loss: 0.1232 - accuracy: 0.9717 - val_loss: 0.1442 - val_accuracy: 0.9515
Epoch 8/1